In [1]:
## 安装langchain和openai
%pip install langchain==0.0.270 openai

<h3>Model I/O</h3>
<h5>1.Prompts</h5>
<h5>2.Language models</h5>
<h5>3.Output parsers</h5>

In [1]:
## load_dotenv 加载环境变量 即当前文件夹下的.env
## 可以添加配置OPENAI_API_KEY="sk-xxx"
import os
# os.environ.clear()
from dotenv import load_dotenv
## 加载环境变量
load_dotenv()

True

<h5>1.1 PromptTemplate</h5>
<h5>1.2 ChatPromptTemplate</h5>

In [2]:
# PromptTemplate 

from langchain import PromptTemplate
prompt_template = PromptTemplate.from_template("please tell me what is {topic}?")
fm= prompt_template.format(topic="ModelScope")
print(fm)

## 显示的指定input_variables
prompt_tp2=PromptTemplate(template="how to use {tool} to work with {app}?", input_variables=["tool","app"])
prompt_tp2.format(tool="langchain", app="Bilibili")


c:\Users\hh\AppData\Local\Programs\Python\Python311\Lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.25) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


please tell me what is ModelScope?


'how to use langchain to work with Bilibili?'

In [3]:
#Chat prompt template
from langchain.prompts import ChatPromptTemplate
tp =ChatPromptTemplate.from_messages([
    ("system","""you are an IT programming expert, when i ask your question you should give the accurate answer.
    Don't make it up.if you don't know, just say i have no idea
    """),
    ("human","{question}")
])
mes= tp.format_messages(
question="how to use langchain to work with Bilibili"
)

from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0,openai_api_key=os.environ.get('OPENAI_API_KEY'), model_name="gpt-3.5-turbo",verbose=True)
# llm(mes)

#SystemMessage HumanMessage
from langchain.prompts.chat import SystemMessage, HumanMessage
tp2 =ChatPromptTemplate.from_messages([
    SystemMessage(content="""you are an IT programming expert, when i ask your question you should give the accurate answer.
    Don't make it up.if you don't know, just say i have no idea
    """),
    HumanMessage(content="{question}")
])
mes2=tp2.format_messages(question="""i want to use python3+langchain to write the automation tool that 
                    can connect with Bitbucket,Jira,Confluence and postman, can give me a solution
                    """)
llm(mes2)

AIMessage(content='What is the difference between front-end and back-end development?', additional_kwargs={}, example=False)

<h5>1.3 CustomPromptyTemplate</h5>

In [39]:
#Define custom prompt template
#https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/custom_prompt_template

import inspect
def get_source_code(function_name):
    #Get the source code of the function
    return inspect.getsource(function_name)


from langchain.prompts import StringPromptTemplate
from pydantic import BaseModel, validator

PROMPT="""\
Given the function name and source code, generate an Chinese language explanation of the function.
Function Name: {function_name}
Source Code:
{source_code}
Explanation:
"""

class FunctionExplainerPromptTemplate(StringPromptTemplate,BaseModel):
    """
    A custom prompt template that takes in the function name as input, 
    and formats the prompt template to provide the source code of the function.
    """

    @validator("input_variables")
    def validate_input_variables(cls, v):
        """
        Validate that the input variables are correct.
        """
        if len(v) !=1 or "function_name" not in v:
            raise ValueError("function_name must be the only input_variable.")
        return v

    def format(self, **kwargs) -> str:
        #Get the source code of the function
        source_code=get_source_code(kwargs["function_name"])

        #Generate the prompt to be sent to the language model
        prompt=PROMPT.format(function_name=kwargs["function_name"].__name__
                             ,source_code=source_code)
        return prompt

    def _prompt_type(self) -> str:
        return "function_explainer"

In [40]:
#use the custom prompt template
f_e=FunctionExplainerPromptTemplate(input_variables=["function_name"])

prompt=f_e.format(function_name=get_source_code)
print(prompt)

Given the function name and source code, generate an Chinese language explanation of the function.
Function Name: get_source_code
Source Code:
def get_source_code(function_name):
    #Get the source code of the function
    return inspect.getsource(function_name)

Explanation:



<h5>1.4 FewShotPromptTemplate</h5>
<font color=#fec4ae size=2>Few-shot prompt 是一种在大型语言模型（LLM）中使用的技术，它可以通过提供少量的示例来帮助模型理解任务并生成相应的内容。 Few-shot prompt 通常用于那些需要模型快速学习新任务或适应新领域的场景，例如文本分类、摘要、翻译、问答等。
在这些场景中，Few-shot prompt 可以通过提供少量的输入-输出示例来帮助模型理解任务，并在不需要大量训练数据或重新训练模型的情况下快速生成相应的内容。这种方法在处理小数据集或需要快速适应新任务的场景中非常有用。</font>

In [41]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate


examples=[
    {
     "question":"what's our team current Sprint JIRA task?",
     "answer":f"JIRA#team current Sprint JIRA list#/v1/jira/team1#GET#param1"
    },
    {
     "question":"what's our team last Sprint JIRA task?",
     "answer":f"JIRA#team current Sprint JIRA list#/v1/jira/team1#GET#param2"
    },
    {
     "question":"what's the William's JIRA task in current Sprint?",
     "answer":f"JIRA#team current Sprint JIRA list#/v1/jira/team1#GET#param3"
    },
    {
     "question":"what's the TODO List JIRA task in current Sprint?",
     "answer":f"JIRA#team current Sprint JIRA list#/v1/jira/team1#GET#param4"
    }
]

In [27]:
e_p=PromptTemplate(input_variables=["question","answer"],template="Question: {question}\n{answer}")
print(e_p.format(**examples[3]))

Question: what's the TODO List JIRA task in current Sprint?
JIRA#team current Sprint JIRA list#/v1/jira/team1#GET#param4


In [42]:
# Feed examples and formatter to FewShotPromptTemplate
# sys_prompt = "do not validate the filed in ‘answer’, which in below examples"

# temp = f"""
# {sys_prompt}
# """
prompt=FewShotPromptTemplate(
    examples=examples,
    example_prompt=e_p,
    suffix="Question: {input}",
    input_variables=["input"]
)
# print(e_p)
print(prompt.format(input="what's the Cancelled List JIRA task in current Sprint?"))

Question: what's our team current Sprint JIRA task?
JIRA#team current Sprint JIRA list#/v1/jira/team1#GET#param1

Question: what's our team last Sprint JIRA task?
JIRA#team current Sprint JIRA list#/v1/jira/team1#GET#param2

Question: what's the William's JIRA task in current Sprint?
JIRA#team current Sprint JIRA list#/v1/jira/team1#GET#param3

Question: what's the TODO List JIRA task in current Sprint?
JIRA#team current Sprint JIRA list#/v1/jira/team1#GET#param4

Question: what's the Cancelled List JIRA task in current Sprint?


In [45]:
#SemanticSimilarityExampleSelector 

from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings


example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(),
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma,
    # This is the number of examples to produce.
    k=1
)

# Select the most similar example to the input.
question = "what's the Cancelled List JIRA task in current Sprint?"
selected_examples = example_selector.select_examples({"question": question})
print(f"Examples most similar to the input: {question}")
for example in selected_examples:
    print("\n")
    for k, v in example.items():
        print(f"{k}: {v}")

<h5>1.5 load_prompt</h5>
<font color=#fec4ae size=2>当将模板和代码分开的时候就可以使用`load_prompt`加载模板文件</font>

`from langchain.prompts import load_prompt`

In [ ]:
# prompt
{
    "_type": "prompt",
    "input_variables": ["adjective", "content"],
    "template": "Tell me a {adjective} joke about {content}."
}

#prompt嵌套其他的template
{
    "_type": "prompt",
    "input_variables": ["adjective", "content"],
    "template_path": "simple_template.txt"
}

#example
[
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"}
]

#few-shot example
{
    "_type": "few_shot",
    "input_variables": ["adjective"],
    "prefix": "Write antonyms for the following words.",
    "example_prompt": {
        "_type": "prompt",
        "input_variables": ["input", "output"],
        "template": "Input: {input}\nOutput: {output}"
    },
    "examples": "examples.json",
    "suffix": "Input: {adjective}\nOutput:"
}

#Template with OutputParser
{
    "input_variables": [
        "question",
        "student_answer"
    ],
    "output_parser": {
        "regex": "(.*?)\\nScore: (.*)",
        "output_keys": [
            "answer",
            "score"
        ],
        "default_output_key": null,
        "_type": "regex_parser"
    },
    "partial_variables": {},
    "template": "Given the following question and student answer, provide a correct answer and score the student answer.\nQuestion: {question}\nStudent Answer: {student_answer}\nCorrect Answer:",
    "template_format": "f-string",
    "validate_template": true,
    "_type": "prompt"
}

